In [ ]:
#class Hom():


Hom = kd.notation.Record("Hom", ("dom", Ob), ("cod", Ob))
Arr = DeclareSort("Arr")
Ob = DeclareSort("Ob")

def HomConst(name, a, b):
    f = Const(name, Arr)
    f.wf =  typ(f, Hom.mk(a,b)) #typeof(f) == Hom(a, b)
    return f




In [ ]:
# explicit tagging
Hom = kd.notation.Record("Hom", ("morph", Morph),("dom", Ob), ("cod", Ob),pred=lambda f: And(x.morph.dom() == x.dom, x.morph.cod() == x.cod))


In [ ]:
# a lightweight overloading with extra metadata.
# like QForAll and QExists but more implicit.

# This ought to kind of be ok because you'll always call Const and it will be a thing you hand to quantifiers?
# we don't need x + y to return a "Nat" because that's find of a proof obligation.?
# as an explicit cast that tags a wf onto that ast?
# but then what do we do with the wf? when do we notice?
# skolem symbols kind of give proof objects.
 
class Nat(IntRef):
    wf = [lambda x: x >= 0]
    def Const(x):
        z = Const(x, IntSort())
        z.wf = z >= 0


def Exists(xs, body):


def ForAll(xs, body):
    wf = []
    for x in xs:
        if hasattr(x, 'wf'):
            wf = += x.wf
    if len(wf) == 0:
        return z3.ForAll(xs,body)
    else:
        return z3.ForAll(xs, z3.Implies(z3.And(wf), body))

In [ ]:
from dataclasses import dataclass

# This isn't really just a Gat. It's hypothetical notions
# sequent?
# it's exploded out just sop we don't have to go digging. Technically, wse could find these bits by digging in pf
@dataclass
class GAT():
    vars: list[ExprRef]
    hyps: list[BoolRef]
    conc: BoolRef #list?
    pf: Proof  # forall vars, hyp => conc. Optional? Things to prove vs things proven

    def __or__(self,other):
        vars = self.vars + other.vars
        hyp =
        conc = 
        return GAT(self.vars + other.vars, self.hyps + other.hyps, And(self.conc,other.conc ), lemma(ForAll(vars, Implies(hyps, concs))  by=[self.pf, other.pf]) )
@dataclass
class CtxTerm():
    vars: list[ExprRef]
    hyps: list[BoolRef]
    wf: BoolRef # wf is analgous to "typechecks"
    t: ExprRef
    pf: Optional[Proof] # forall vars, Implies(hyps, wf(t)))

    # so it's a Sequent of well formedness + a term

    def __eq__(self, other, by=[]):

    def formula(self):
        return ForAll([self.vars], Implies(hyps, self.wf())
    def check(self):
        # sanity checking. not really part of trusted code. Arguable.
        self.formula().eq(pf.thm)
        
@dataclass
class CtxTerm(Sequent):
    t: ExprRef


class Hom(CtxTerm):
    def comp(f : CtxTerm,g):
        comp = Function("comp", Hom, Hom, Hom)
        Hom(  comp(f.t, g.t), lemma( cod(f.t) == dom(g.t))


class Nat(CtxTerm):

    def Const(name):
        x = Int(name)
        return CtxTerm(x, x >= 0, x >= 0, lemma(ForAll([x], Implies(x >= 0, x >= 0))), x)


Yeah, what about that yoneda thing. Makes things pointfulk right? Preapplication with compose

f = comp(f,_)



# GATs
generalized algerbaic theories
https://www.epatters.org/wiki/algebra/dependently-typed-algebraic-theories
eseentially algerbraic theories
second order

https://arxiv.org/abs/1902.08848 Algebraic Type Theory and Universe Hierarchies

twee handles horn ok... What about just using twee?




Functor, Nat Trans, Kan, Adjunction. These were harder.
ASP for finite categories. Remember that one?

https://www.jstor.org/stable/2275472 Axiomatiizng a category of categories - colin mclarty


https://mathoverflow.net/questions/9269/category-of-categories-as-a-foundation-of-mathematics

I dunno man.

categorica mathematica 

F-algebra. It was a generic way of talking about
cata()





In [ ]:
CAT = DeclareSort("CAT")
Functor = DeclareSort("Functor") #ArraySort(CAT, CAT) # Is it ok to have functors as "functions"

cod = Function('cod', Functor, CAT)
dom = Function('dom', Functor, CAT)




NatTrans = ArraySort(Functor, Functor)

From old notes
Topos Book
Scott Encoding using exists predicate.
Well formed as precondition.

In [ ]:
Arr = DeclareSort("Arr")
Ob = DeclareSort("Ob")
ex = Function("Ex", Arr, BoolSort())

f,g,h = Consts("f g h", Arr)
A,B,C = Consts("A B C", Ob)
comp = Function("comp", Arr, Arr, Arr)
id_ = Function("id", Ob, Arr)
dom = Function("dom", Arr, Ob)
cod = Function("cod", Arr, Ob)
z3.ExprRef.__lshift__ = lambda self, other: comp(self, other)
#ax_comp = trust(ForAll([f,g,h], comp(f,comp(g,h)) == comp(comp(f,g),h)))
ax_comp_assoc = trust(ForAll([f,g,h], f << (g << h) == (f << g) << h))
#ax_id_l = trust(ForAll([f,A], (cod(f) == A) > (id_(A) << f == f)))
#ax_id_r = trust(ForAll([f,A], (dom(f) == A) > (f << id_(A) == f)))
ax_id_l = trust(ForAll([f], id_(cod(f)) << f == f)) # inline the dom/cod
ax_id_r = trust(ForAll([f], f << id_(dom(f)) == f)) 
ax_cat = [ax_id_l, ax_id_r, ax_comp_assoc]

ax_dom_id = trust(ForAll([A], dom(id_(A)) == A))
ax_cod_id = trust(ForAll([A], cod(id_(A)) == A))
ax_ex_id = trust(ForAll([A], ex(id_(A))))
ax_ex_comp = trust(ForAll([f,g], ex(f << g) == (dom(f) == cod(g))))
ax_wff = [ax_dom_id, ax_cod_id, ax_ex_id, ax_ex_comp]

junk = Const("junk", Arr)
ax_junk = trust(ForAll([f], Implies(ex(f), f != junk))) # bidirectional?

wff_id_id = infer(ax_wff + ax_cat, ex(id_(A) << id_(A)) & (dom(id_(A) << id_(A)) == A) & (cod(id_(A) << id_(A)) == A))
wff_id_id
#infer(ax_cat + ax_wff + [wff_id_id], id_(A) << id_(A) == id_(A))

# sanity check
#infer(ax_cat + ax_wff, False)

monic, monic_def = define("monic", Lambda([f], ForAll([g,h], Implies(f << g == f << h, h == g))))
epic, epic_def = define("epic", Lambda([f], ForAll([g,h], Implies(g << f == h << f, h == g))))

id_monic = infer(ax_cat + ax_wff + [monic_def], ForAll([A], monic[id_(A)]))
id_monic
# pullback is universal square 
pullback = define("pullback", Lambda([f,g,h,k, univ], 
f << g == h << k & 
ForAll([x,y], Exists([univ], Implies(x << g == y << k, x == univ << g  & y == univ << k)))))

infer(ax_cat + ax_wff + [monic_def], False)

https://www.youtube.com/watch?v=c3qfeSmhN-8&ab_channel=ToposInstitute 
Christoph Benzmueller: "Logico-pluralistic exploration of foundational theories with computers"
https://www.youtube.com/watch?v=XaxCeVOMfQ4&ab_channel=AlianzaenInteligenciaArtificial


A domain of existing/well defined things as a subet of a larger domain

Scott 1967 Dana Scott, Existence and description in formal logic
Scott 1977 Identity and existence in intuitionistic logic

Dewcription operators


Bill Farmer, Imps and his book. He had some acronym for a notion of partial existence equality

This is in tptp https://www.philipzucker.com/notes/Logic/tptp/#cat003-0ax

https://plato.stanford.edu/entries/logic-free/

Kind of the point of egglog was for categorical stuff.
Free logic is possibly appropriate for GATs, Essentially algebraic theories, partial horn logic.

Encoding free logic is a lot like the standard interpretation of modal logic. 

Generalized monoids

https://www.philipzucker.com/category-theory-in-the-e-automated-theorem-prover/ here I use a relational implementation of compose.


In [ ]:
Ex(x) == Edom(x)
Ex(x) == Ecod(x)
Ex(comp(x, y)) == (dom(x) == cod(y))
kleene(comp(x,comp(y,z)), comp(comp(x,y),z))
kleene(comp(x, dom(x)),  x)
kleene(comp(cod(x), x), x)

In [ ]:
strict = Implies(Ex(comp(x,y)), And(Ex(x), Ex(y))
exists = Implies() 
assoc = 

def ID(i):
    FForAll(x, Implies(Ex(i), comp(i, x) )

In [ ]:
# This is an equivalnce relation
def kleene(x,y):
    Implies(And(Ex(x), Ex(y)), x == y)

# existing identity. transitive and symmetric but not reflexive. undefined elements are not self exists equal
def ex_eq(x,y):
    return And(Ex(x), Ex(y), x == y)

# 1/0 is not ex_eq with itself.

def FExists(x,P):
    return Exists(x, And(Ex(x), P))
def FForAll(x,P):
    return ForAll(x, Implies(Ex(x), P))


In [ ]:
Ex = Function('Ex', S, B)

# but they did a deeper encoindg
#Ex = Function("Ex", ArraySort())
# nope.

junk = Const('junk', S) # they call it *

# hmm. He's using isabelle's THE, but I don't have one?
the = Function('the', ArraySort(S,B), S)

desc = Function('desc', , S)
desc[phi] == If(FExists(x, And(phi[x], FForAll([y], Implies(phi(y), x = y)) )), the(And(Ex(x), phi(x))), junk

Interesting notions of equality. They have 3 distinct ones.
Extensional equality eq()

iso(a = b, iso(a,b,f,g), p, q). univalence. iso are permutations. fraenkel mostowski is permutations. Any relation?
parametricity is about abstraction hiding. wadler's girard reynolds had something about Nats being isomorphic

alpha_eq / nominal_eq

A proof relevant form of
eq(a = b, eq(a,b))

Observational equality.

a = b <-> eq(P, a , b) (aka  typeof(P, eq(a,b))  / proof(P, eq(a,b)))
z3 won't be happy. Remmber when I was trying to use `=`. Once you're eq you're kind of screwed because refl becomes the proof.

This is a separate issue from transitive closure?

Using predicates to cut out subdomains:
- nbg separating classes from sets
- free logic, good from bad terms
- atoms from sets in Franekl Mostowski
- maybe kripke / seperation / intuitionsitc encodings?

def ForAllSet(x, P):
    return ForAll([x], Implies(Set(x), P))
def ExistsSet():




In [ ]:
# system T
Nat = Datatype("Nat") 
#...
def induct(T):
    # T is a Combo of ArraySort and Nat
    F = FreshConst("F", T)
    x = FreshConst("x", Nat)
    return ForAll([F], Implies(And(F(Nat.zero) == Nat.zero, ForAll([x], Implies(F(x) == Nat.zero, F(Nat.succ(x)) == Nat.zero))),
                               ForAll([x], F(x) == Nat.zero)))
# f(x) == zero is arepresentation of truthhood.

# system F



In [ ]:
ctx = [x,y,z]
def lemma(thm,by=[]): # ctx=ctx
    global ctx
    #prove(ForAll(ctx, Implies(And(*by), thm)))
    #  no actually, this is probably what I want.
    prove(Implies(And(*by), ForAll(ctx, thm)))
    return ForAll([x], thm)
#Or just define new lemma, every time I enter new context
# I like this. Avoids a lot of boilerplate.
# I don't think I'll ever be "wrong". I just may not be doing something I don't expect
# mutate ctx variable later.
# also, now countermodels with more information may be possible.


In [ ]:
from z3 import *
Ob = DeclareType("Ob")
Hom = DeclareType("Hom")

dom = Function("dom", Hom, Ob)
cod = Function("cod", Hom, Ob)

E = Function("E", Hom, Bool) # well formed
comp = Function("comp", Hom, Hom, Hom)
id = Function("id", Ob, Hom)


def keq(x,y):
    return Implies(And(E(x), E(y)), x == y)
class Hom(ExprRef):
    def __eq__(self, other):
        # kleene equality
        return Implies(And(E(self), E(other)), self == other)

wf_comp = ForAll([x,y], Implies(And(E(x), E(y), cod(x) == dom(x)), E(comp(x,y))))

keq_comp = ForAll([a], 